In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2



import pandas as pd
import io
import boto3
from boto3.dynamodb.conditions import Key
import os
from google_images_download import google_images_download
import glob
import PIL
import json
import numpy as np
import pathlib
import pickle

import textacy
import en_core_web_sm

from IPython.display import SVG

from IPython.display import Image
 

# Create Training data for text to image model

## Inputs

In [22]:
tblName = "ResourceDocuments"
nodeIdentifierName = "photosynthesis-photosynthesis-photosynthesis-biology"

termTxtToImage_flpth = 'data/photosynthesis'
text_flpth = os.path.join(termTxtToImage_flpth, 'text')
img_flpth =  os.path.join(termTxtToImage_flpth, 'images')
imageLog_fir='logs'

resourceDbName = 'dynamodb'
s3Bucket = "egm-bucket/TEXT_TO_IMAGE_DATA/data"

### Get Data from Resource Db: 
photosynthesis whole

In [23]:
# Get Definitions for photosynthesis from dynamodb

## Connect to dynamodb
dynamodbClient = boto3.resource("dynamodb")
# client = boto3.client('dynamodb')
# display(client.describe_table(TableName=tblName))

## Connect to table with resources
resourceTbl = dynamodbClient.Table(tblName)
# display(resourceTbl.global_secondary_indexes)
display("NUmber of Items in ResourceDb: {}".format(resourceTbl.item_count))

'NUmber of Items in ResourceDb: 14'

### Load text into pandas 
- For data munging
    - stats
    - Duplicates

In [24]:
response = resourceTbl.query(
    IndexName='NODE_IDENTIFIER-index',
    KeyConditionExpression=Key('NODE_IDENTIFIER').eq(nodeIdentifierName)
)

# Pass through pandas for some data munging
rsrc_df = pd.DataFrame(response["Items"])
print("Db Response Shape: {}".format(rsrc_df.shape))

rsrc_df.drop_duplicates(['RESOURCE'], keep='last', inplace=True)
rsrc_df.reset_index(drop=True)

print("Db Response Shape: {}".format(rsrc_df.shape))
print(rsrc_df.columns)

Db Response Shape: (14, 13)
Db Response Shape: (7, 13)
Index(['IMAGES', 'NODE_IDENTIFIER', 'POS', 'RESOURCE', 'RESOURCE_ATTRIBUTION',
       'RESOURCE_DATATYPE', 'RESOURCE_SOURCE', 'RESOURCE_TYPE', 'RESOURCE_URL',
       'TERM', 'TIME_DOWNLOADED', 'TOPIC', 'UNIQUE_IDENTIFIER'],
      dtype='object')


In [25]:
rsrc_df["RESOURCE_SOURCE"]

4        century
5        wordnet
9     wiktionary
10     wikipedia
11         gcide
12     wikipedia
13    ahd-legacy
Name: RESOURCE_SOURCE, dtype: object

### Create Labels for text to image model

In [ ]:
oxf = "The process by which green plants and some other organisms use sunlight to synthesize nutrients from carbon dioxide and water. Photosynthesis in plants generally involves the green pigment chlorophyll and generates oxygen as a by-product."


In [ ]:
rsrc_df.append(pd.DataFrame({"RESOURCE_SOURCE": [oxf]}, sort=False))

In [82]:
def handle_missing_directories(directory_flpth):
    # Handle missing Directory
    if not os.path.exists(directory_flpth):
        
        os.makedirs(directory_flpth)
        print("Made new directory: {}".format(directory_flpth))
        # print(os.path.join(dirname, flpth))
    else:
        pass
    
    return

# Create text file for each doc - Each Doc maps to an image

## TODO: incoroporate number of labals per line
def labels_to_imageTxt_files(rsrc_df, trainingData_term, trainigData_flpth='../data'):
    
    # Handle if a data directory for a term exists e.g. data/photosynthesis
    dirname = os.path.abspath('')
    termData_flpth = os.path.join(dirname, trainigData_flpth)
    handle_missing_directories(termData_flpth)
    
    
    ### Move resource df to textacy
   
    # Load into textacy to delimit sentences
    img_labels = rsrc_df.to_dict(orient="records")
    text_stream, metadata_stream = textacy.io.split_records(img_labels, 'RESOURCE')

    # Load english model
    en = en_core_web_sm.load()
    labels_corpus = textacy.Corpus(lang=en, texts=text_stream, metadatas=metadata_stream)
    
    caption_filename_path = os.path.join(trainigData_flpth, "captions.pickle")
    
    # Loop through corpus and write document to flpth (s3)
    ''' Each doc in a corpus equals and image'''
    for ix, doc in enumerate(labels_corpus):
        print("Number of Sentences: {}".format(doc.n_sents))
        
        # Paths to directories (Where to write the text files)
        filename = "{}_{}.txt".format(trainingData_term, ix)
        path_to_file = "{}/{}".format(trainigData_flpth, filename)
        
        # Write captions for google images
        f =  open(path_to_file, 'w')
        
        # Parse Document into sentences
        for sent in doc.sents:
            caption = textacy.preprocess.preprocess_text(sent.text,
                                               lowercase=True,
                                               no_punct=True
                                              )
            # f.write(label+"\n" )
            f.write(caption+" " )
            
        f.close()
        
    return ix + 1 # Count using 1 as start

# process labels for images


trainingData_term = 'photosynthesis'
txt_trainingData_flpth='{}/text'.format(termTxtToImage_flpth, 'text')

numText_files = labels_to_imageTxt_files(rsrc_df, trainingData_term, txt_trainingData_flpth)
numText_files

Number of Sentenses: 1
Number of Sentenses: 1
Number of Sentenses: 1
Number of Sentenses: 16
Number of Sentenses: 9
Number of Sentenses: 5
Number of Sentenses: 2


7

In [ ]:
def get_relfilename(fullFlpth, relFlpth):
    p = pathlib.Path(fullFlpth)
    txtRel_flpth = p.relative_to(relFlpth)
    txtCaption_flpth = str(txtRel_flpth.parent)
    
    txtFile_nm = txtRel_flpth.stem
    
    return txtFile_nm

In [ ]:
orgFileName = get_relfilename(fileCombo_lst[0], )

In [ ]:
# Create Fake data
import shutil
import itertools

# Get a list of the original files f
txtDirFileNm_lst = glob.glob(text_flpth+"/**/*.txt", recursive=True)
imgDirFileNm_lst = glob.glob(img_flpth+"/**/*.jpg", recursive=True)

fileCombo_lst = list(itertools.product(txtDirFileNm_lst,imgDirFileNm_lst))

# for item in fileCombo_lst:
    
#     # Get text name
#     txtget_relfilename

# # # Create list paired tuple pairs
# # comboFlpth_lst = list(zip(txtDirFileNm_lst, imgDirFileNm_lst))
# # comboFlpth_lst

# # new_list = []
# # for k,v in comboFlpth_lst.iteritems():
# #     new_list.extend([x for x in combinations(v, 2)]) 
# # # for txtFile, imgFile in comboFlpth_dict:
# # #     # Make copies of files and rename
# # #     shutil.copyfile(txtFile, dst)
# # #     shutil.copyfile(imgFile, dst)

# # new_list



### Download Images from google 

In [21]:

def download_images(term ,img_args):
    
    # Download Images 
    response = google_images_download.googleimagesdownload()
    img_paths = response.download(img_args)
    
#     # Post Process google image results
#     for idx, f in enumerate(img_paths[term]):
        
#         # Open Google image resulst and conver to jpeg
#         img = PIL.Image.open(f)
#         img_filetype = img.format.lower()  # 'JPEG'
        
#         rgb_img = img.convert('RGB')
#         img.close()
        
#         # Make new filenme to allign with text file name
#         filename = "{}_{}.{}".format(trainingData_term, idx, 'jpg')
#         newfilepath_f = os.path.join(os.path.dirname(f), filename)
        
#         # Save and image
#         rgb_img.save(newfilepath_f)
#         os.remove(f)
    
    
    return response 

img_args = {"keywords":"sun",
             "format": "png",
              "limit": 100,
             "output_directory": 'data',
            "metadata": True,
            "image_directory": "photosynthesis/images",
            "no_download": False,
            "extract_metadata":True
            # "size":"icon"
           }

response = download_images(trainingData_term, img_args)

NameError: name 'trainingData_term' is not defined

### 1. Inputs

In [11]:
import tigDataLoader.utiils as dataloader
import os
import json

In [137]:
predictTerm = "photosynthesis"
dataTerm = "photosynthesis"

In [138]:
# Base file io inputs
trainData_flpth = os.path.join('data', predictTerm)
text_flpth = os.path.join(trainData_flpth, 'text')
img_flpth =  os.path.join(trainData_flpth, 'images')

# Google imagae download metadata
imageLog_fir='logs'

In [139]:
def delete_images():
    return

In [52]:
def transformText_to_captions(txtDoc, numCaptions_per_image=5, txtSplit_method='svo'):
    # Take textacy doc and converts to a list of captions for an image
    
    if txtSplit_method=='noun_chunks':
        split_list = list(textacy.extract.noun_chunks(txtDoc, drop_determiners=False, min_freq=1))
    if txtSplit_method=='svo':
        split_list = list(textacy.extract.subject_verb_object_triples(txtDoc))
    
    return split_list

### 2. Make captions for images

In [30]:
# 1. Load text to textacy - each document is linked to an image
imgTxt_corpus = dataloader.df_to_corpus(rsrc_df)
imgTxt_corpus

Corpus(7 docs; 963 tokens)

In [75]:
imgTxt_corpus.word_doc_freqs(normalize='lemma', weighting='count', as_strings=True)

{'bring': 1,
 'light': 6,
 'combination': 1,
 'sunlight': 1,
 'action': 2,
 'carbohydrate': 6,
 'water': 6,
 'air': 2,
 'chemical': 3,
 'live': 1,
 'formation': 1,
 'di': 1,
 'plant': 7,
 'influence': 1,
 'oxid': 1,
 'carbon': 6,
 'synthesis': 3,
 'aid': 1,
 'especially': 1,
 'energy': 5,
 'compound': 3,
 'radiant': 1,
 'photoautotroph': 3,
 'chloroplast': 3,
 'dioxide': 5,
 'generate': 1,
 'process': 5,
 'oxygen': 4,
 'embed': 1,
 'oxygenation': 1,
 'transfer': 1,
 'phosphate': 1,
 'remove': 1,
 'release': 3,
 'storage': 1,
 'φῶς': 1,
 'appear': 2,
 'nicotinamide': 1,
 'achieve': 1,
 'evolution': 1,
 'term': 1,
 '91': 1,
 'organic': 2,
 'reverse': 1,
 'ribulose': 1,
 'petagram': 1,
 'cycle': 1,
 'excess': 1,
 'reaction': 1,
 'bacteria': 1,
 'use': 4,
 'history': 1,
 'current': 1,
 'serve': 1,
 'reduce': 2,
 'largely': 2,
 'life': 2,
 'hydrogen': 2,
 'sugar': 3,
 'average': 1,
 'leaf': 1,
 'dinucleotide': 1,
 'organism': 3,
 'sulfide': 1,
 'responsible': 2,
 'cyanobacteria': 2,
 'today

In [60]:
txtDoc = imgTxt_corpus[0]
txtDoc

Doc(37 tokens; "A chemical combination brought about by the act...")

In [81]:
imgTxt_corpus


Corpus(7 docs; 963 tokens)

In [ ]:
imgTxt_corpus.get(lambda x: len(x)

In [56]:
docSplit_list = transformText_to_captions(txtDoc, numCaptions_per_image=5, txtSplit_method='svo')

In [70]:
list(textacy.extract.ngrams(txtDoc, n=5, filter_stops=False))

[A chemical combination brought about,
 chemical combination brought about by,
 combination brought about by the,
 brought about by the action,
 about by the action of,
 by the action of light,
 as in the formation of,
 in the formation of carbohydrates,
 the formation of carbohydrates in,
 formation of carbohydrates in living,
 of carbohydrates in living plants,
 carbohydrates in living plants from,
 in living plants from the,
 living plants from the carbon,
 plants from the carbon di,
 oxid and water of the,
 and water of the air,
 water of the air under,
 of the air under the,
 the air under the influence,
 air under the influence of,
 under the influence of sunlight]

In [68]:
txtDoc.text

'A chemical combination brought about by the action of light, as in the formation of carbohydrates in living plants from the carbon di-oxid and water of the air under the influence of sunlight.'

In [71]:
textacy.extract.acronyms_and_definitions(txtDoc)

{}

### 3. Download images
- from google

In [18]:
img_args = {"keywords": dataTerm,
             "format": "png",
              "limit": 100,
             "output_directory": 'data',
            "metadata": True,
            "image_directory": "{}/images/".format(predictTerm),
            "no_download": False,
            "extract_metadata":True,
            "type": "clipart"
           }
img_paths, response = dataloader.download_images(dataTerm, img_args)


Item no.: 1 --> Item name = sun
Evaluating...
Starting Download...

Image Metadata: {'image_format': 'png', 'image_height': 2358, 'image_width': 2361, 'image_link': 'https://www.bedfordny.gov/wp-content/uploads/2018/07/sunshine-sun-clip-art-with-transparent-background-free-free-clipart-sun-2361_2358-2.png', 'image_description': 'sunshine-sun-clip-art-with-transparent-background-free-free-clipart ...', 'image_host': 'bedfordny.gov', 'image_source': 'https://www.bedfordny.gov/community-events-calendar-6-30-7-7-2018/sunshine-sun-clip-art-with-transparent-background-free-free-clipart-sun-2361_2358-3/', 'image_thumbnail_url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTO8VPsoy8d-O_GGnT_HsgEJrWr8OFlrpDUM9Z1SZKpwm0AX7zx'}
URLError on an image...trying next one... Error: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:852)>

Image Metadata: {'image_format': 'jpg', 'image_height': 400, 'image_width': 400, 'image_link': 'http://worldartsme.com/image


Item no.: 1 --> Item name = sun
Evaluating...
Starting Download...

Image Metadata: {'image_format': 'png', 'image_height': 345, 'image_width': 365, 'image_link': 'https://solarsystem.nasa.gov/system/basic_html_elements/11561_Sun.png', 'image_description': 'Overview | Sun – Solar System Exploration: NASA Science', 'image_host': 'solarsystem.nasa.gov', 'image_source': 'https://solarsystem.nasa.gov/solar-system/sun/overview/', 'image_thumbnail_url': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQIfOTyp0dLP0GwmyB7i4NdsZMl0uQYvxk8wVY43iGMsAUKXVG4'}
Completed Image ====> 1. 11561_sun.png

Image Metadata: {'image_format': 'png', 'image_height': 336, 'image_width': 500, 'image_link': 'https://spaceplace.nasa.gov/review/sunburn/sunburn1.en.png', 'image_description': 'Why does the sun burn us? | NASA Space Place – NASA Science for Kids', 'image_host': 'spaceplace.nasa.gov', 'image_source': 'https://spaceplace.nasa.gov/sunburn/', 'image_thumbnail_url': 'https://encrypted-tbn0.gstatic.c

In [ ]:
## load text into textacy
imgTxt_corpus = df_to_corpus()


In [19]:
## Load image meta-data json 
with open('coco/logs/sun.json') as f:
    data = pickle.load(f)
    
data

In [143]:
with open("data/photosynthesis/test/filenames.pickle", "rb") as pickF:
    tstFilenames = pickle.load(pickF)
    
display(type(tstFilenames),
        len(tstFilenames),
        tstFilenames)

list

5

['text/photosynthesis_5',
 'text/photosynthesis_6',
 'text/photosynthesis_3',
 'text/photosynthesis_1',
 'text/photosynthesis_2']

In [144]:
with open("data/photosynthesis/train/filenames.pickle", "rb") as pickF:
    trFilenames = pickle.load(pickF)
    
display(type(trFilenames),
        len(trFilenames),
        trFilenames)

list

5

['text/photosynthesis_4',
 'text/photosynthesis_0',
 'text/photosynthesis_5',
 'text/photosynthesis_6',
 'text/photosynthesis_3']

In [115]:
commonalities = set(trFilenames) - (set(trFilenames) - set(tstFilenames))
commonalities

set()

In [94]:
def txtFilenamesTo_pickle(flpth, lst_to_write):
    
    handle_missing_directories(os.path.dirname(flpth))
    
    pickle_out  = open(flpth, "wb")
    pickle.dump(lst_to_write, pickle_out)
    pickle_out.close()

    return 
    
def splitData(trainSplit, testSplit, filename_lst):
    
    # Calculate total number of filenames
    num_filenames = len(filename_lst)

    numTrain_files = np.ceil(trainSplit * num_filenames).astype(int)
    numTest_files = np.floor(testSplit * num_filenames).astype(int)
    
    print("Number of Train files: {}".format(numTrain_files))
    print("Number of Test files: {}".format(numTest_files))
    
    trainFile_lst = filename_lst[:numTrain_files]
    testFile_lst = filename_lst[-numTrain_files:]
    
    return {
        "train": trainFile_lst,
        "test": testFile_lst
    }



In [145]:
def modified_txt_flpth(full_flpth):
    # Read in filepath and seperate
    p = pathlib.Path(full_flpth)
    txtRel_flpth = p.relative_to(termTxtToImage_flpth)
    txtCaption_flpth = str(txtRel_flpth.parent)
    
    txtFile_nm = txtRel_flpth.stem
    #print("Filename: {}".format(txtFile_nm))    
    
    
    caption_flpth = os.path.join(txtCaption_flpth, txtFile_nm)
    
    return  caption_flpth
# captionFilename = modified_txt_flpth('data/photosynthesis/text/photosynthesis_4.txt')
# captionFilename

### Split Data for model training

In [136]:
# Define Text data directories
text_flpth = os.path.join(termTxtToImage_flpth, 'text')
print("Text Data Directory: {}\n".format(text_flpth))

# Get all the file names from the text directory
captionsFilename_lst = glob.glob(text_flpth+"/**/*.txt", recursive=True)
modifiedCaptionsFilename_lst = [modified_txt_flpth(relCaption_flpth)  for relCaption_flpth in captionsFilename_lst]
modifiedCaptionsFilename_lst

Text Data Directory: data/photosynthesis/text



['photosynthesis_4',
 'photosynthesis_0',
 'photosynthesis_5',
 'photosynthesis_6',
 'photosynthesis_3',
 'photosynthesis_1',
 'photosynthesis_2']

In [133]:
# Write filename to data term directory


In [147]:
display(captionsFilename_lst)
print("Number of files: {}".format(len(modifiedCaptionsFilename_lst)))


# Split the data into training and test
## Will need to accomodate term weightings and try different cossvalidation methods
trainSplit = 1
testSplit = 1

split_dict = splitData(trainSplit, testSplit, modifiedCaptionsFilename_lst)

# write files names to text and pickle
for splitType in split_dict:
    txtFlpth_pickle = os.path.join(termTxtToImage_flpth , splitType, 'filenames.pickle')
    print('Pickling: {} '.format(txtFlpth_pickle))

    txtFilenamesTo_pickle(txtFlpth_pickle, split_dict[splitType])

['data/photosynthesis/text/photosynthesis_4.txt',
 'data/photosynthesis/text/photosynthesis_0.txt',
 'data/photosynthesis/text/photosynthesis_5.txt',
 'data/photosynthesis/text/photosynthesis_6.txt',
 'data/photosynthesis/text/photosynthesis_3.txt',
 'data/photosynthesis/text/photosynthesis_1.txt',
 'data/photosynthesis/text/photosynthesis_2.txt']

Number of files: 7
Number of Train files: 7
Number of Test files: 7
Pickling: data/photosynthesis/train/filenames.pickle 
Pickling: data/photosynthesis/test/filenames.pickle 


#### Split the data into training and test and 
Notes:
    - I think in the original AttnGAN code `test` means `cross-validation`

In [127]:

split_dict[splitType]

['photosynthesis_4',
 'photosynthesis_0',
 'photosynthesis_5',
 'photosynthesis_6',
 'photosynthesis_3',
 'photosynthesis_1',
 'photosynthesis_2']

### Write a text file for all of the caption filenames

In [ ]:
# Get all the file names from the text directory
captions_filename_lst = glob.glob(text_flpth+"/**/*.txt", recursive=True)

In [ ]:
captions_filename_lst

In [ ]:
# Make a text file with a list of the caption filenames

text_flpth = os.path.join(termTxtToImage_flpth, 'text')
print("Text Data Directory: {}\n".format(text_flpth))

# Find all the files with captions in the text directory and write there names to a file
txtFile_lst = glob.glob(text_flpth+"/**/*.txt", recursive=True)

writeFilename = "{}.txt".format('filenames')
writeFilenames_flpth = os.path.join(termTxtToImage_flpth, writeFilename)


# Write list to file in a form ATTN GAN accepts
f =  open(writeFilenames_flpth, 'w')

for item in txtFile_lst:
    
    
    caption_flpth = modified_txt_flpth(item) 
        
    f.write("{}\n".format(caption_flpth))
    print (caption_flpth)
    
f.close()



In [ ]:


pickle.load(termTxtToImage_flpth+"captions")


In [5]:
predict_str =  "The process by which green plants and some other organisms use sunlight to synthesize nutrients from carbon dioxide and water. Photosynthesis in plants generally involves the green pigment chlorophyll and generates oxygen as a by-product."
textacy.preprocess_text(predict_str, lowercase=True, no_punct=True)

'the process by which green plants and some other organisms use sunlight to synthesize nutrients from carbon dioxide and water photosynthesis in plants generally involves the green pigment chlorophyll and generates oxygen as a by product'

In [ ]:
p.stem

In [ ]:

txtRel_flpth = p.relative_to(termTxtToImage_flpth)
txtRel_flpth

In [ ]:
p.stem

In [ ]:
fileParts = txtRel_flpth.parts
fileParts.

In [ ]:
os.path.dirname("/home/ec2-user/environment/AttnGAN/data/photosynthesis/images/5. photosynthesis.png")

In [ ]:
 
s3Client = boto3.client("s3")     
s3Client.Object('my-bucket-name', 'newfile.txt').put(Body=content)

In [ ]:
list(labels_corpus.docs[0].sents)

In [ ]:
import requests
r = requests.get("https://ssec.si.edu/stemvisions-blog/what-photosynthesis")

In [ ]:
r.content